## ***Fake News Classifier Using LSTM***

In [1]:
## ! unzip "/content/drive/MyDrive/Fake News Datasets/fake-news.zip" -d "/content/drive/MyDrive/Fake News Datasets"

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Fake News Datasets/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
## drop NA value
df = df.dropna()

In [7]:
X =df.drop('label', axis = 1)
y =  df['label']

In [8]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [9]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU, Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
## vocaulary size
voc_size = 5000

## **OneHot Representation**

In [12]:
messages = X.copy()

In [13]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
messages.reset_index(inplace = True)

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
## data preprocessing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stopword  = set(stopwords.words('english'))

corpus = []

for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z0-9]',  ' ',  messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [lemmatizer.lemmatize(word) for word in review if not word in stopword]
  review = ' '.join(review)
  corpus.append(review)

In [19]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 '15 civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor p

In [20]:
corpus[1]

'flynn hillary clinton big woman campus breitbart'

In [21]:
voc_size

5000

In [22]:
one_hot_rep = [one_hot(words, voc_size) for words in corpus]
one_hot_rep

[[1346, 3188, 1685, 592, 3946, 2557, 50, 4141, 1813, 444],
 [4474, 1778, 3172, 3195, 3785, 3102, 1683],
 [3889, 3544, 239, 338],
 [4399, 4744, 4977, 3614, 3292, 4173, 79],
 [3511, 3785, 775, 4883, 1586, 1778, 3785, 1221, 2282, 3801],
 [4385,
  4020,
  967,
  2809,
  2819,
  3732,
  4345,
  4799,
  619,
  3931,
  531,
  769,
  2095,
  1487,
  1683],
 [2320, 310, 3496, 3075, 3368, 3877, 1422, 3993, 3511, 2646, 2178],
 [2101, 3150, 4876, 2048, 2797, 4817, 3732, 2322, 3511, 2646, 2178],
 [2097, 3423, 3436, 3695, 118, 233, 3809, 1399, 3732, 3073],
 [3228, 3713, 1489, 4743, 4925, 4903, 270, 130],
 [2335, 2256, 1571, 2405, 909, 3535, 4875, 1105, 2509, 4148, 3047],
 [3292, 4036, 3946, 3569, 3732, 2797],
 [4808, 124, 2551, 4077, 374, 3321, 873, 2423, 2954],
 [672, 16, 2731, 3829, 4503, 3251, 1050, 3511, 2646, 2178],
 [1585, 396, 4195, 53, 4257, 3511, 2646, 2178],
 [2705, 664, 1918, 1466, 2178, 3346, 3264, 2330, 961, 1347, 3792, 2971],
 [999, 3699, 1778],
 [1898, 1783, 4522, 269, 3732, 833, 3556

In [23]:
corpus[1]

'flynn hillary clinton big woman campus breitbart'

In [24]:
one_hot_rep[1]

[4474, 1778, 3172, 3195, 3785, 3102, 1683]

# ***Embadding Representation***

In [25]:
sent_length = 25
embedded_doc = pad_sequences(one_hot_rep, padding = 'post', maxlen = sent_length)
embedded_doc

array([[1346, 3188, 1685, ...,    0,    0,    0],
       [4474, 1778, 3172, ...,    0,    0,    0],
       [3889, 3544,  239, ...,    0,    0,    0],
       ...,
       [3023, 3250, 1460, ...,    0,    0,    0],
       [4773, 2797, 3843, ...,    0,    0,    0],
       [1956, 3919, 4993, ...,    0,    0,    0]], dtype=int32)

In [28]:
from tensorflow.keras.regularizers import l2

In [51]:
## creating model
embedding_vec_feature =1000

model = Sequential()
model.add(Embedding(voc_size, embedding_vec_feature, input_length = sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(101, return_sequences = True, kernel_regularizer = l2(0.01)) ))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(101, return_sequences = True, kernel_regularizer = l2(0.01))))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(101)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 25, 1000)          5000000   
                                                                 
 dropout_29 (Dropout)        (None, 25, 1000)          0         
                                                                 
 bidirectional_27 (Bidirect  (None, 25, 202)           890416    
 ional)                                                          
                                                                 
 dropout_30 (Dropout)        (None, 25, 202)           0         
                                                                 
 bidirectional_28 (Bidirect  (None, 25, 202)           245632    
 ional)                                                          
                                                                 
 dropout_31 (Dropout)        (None, 25, 202)         

In [52]:
len(embedded_doc), y.shape

(18285, (18285,))

In [53]:
import numpy as np
X_final = np.array(embedded_doc)
y_final = np.array(y)

In [54]:
X_final.shape, y_final.shape

((18285, 25), (18285,))

# ***Model Training***

In [55]:
model.fit(X_final, y_final,  validation_split= 0.15, epochs = 100, batch_size =64, callbacks= EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights = True))

Epoch 1/100
243/243 [==============================] - 24s 58ms/step - loss: 1.8793 - accuracy: 0.8820 - val_loss: 0.2320 - val_accuracy: 0.9194
Epoch 2/100
243/243 [==============================] - 5s 23ms/step - loss: 0.1690 - accuracy: 0.9438 - val_loss: 0.2765 - val_accuracy: 0.9107
Epoch 3/100
243/243 [==============================] - 6s 24ms/step - loss: 0.1341 - accuracy: 0.9588 - val_loss: 0.2649 - val_accuracy: 0.9169
Epoch 4/100
243/243 [==============================] - 5s 20ms/step - loss: 0.1203 - accuracy: 0.9672 - val_loss: 0.2535 - val_accuracy: 0.9118


# ***Prediction on Test Dataset***

In [56]:
test_data  = pd.read_csv('/content/drive/MyDrive/Fake News Datasets/test.csv')
test_data

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [57]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [58]:
test_data = test_data.fillna('NA')

In [59]:
test_data.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [60]:
test_data.reset_index(inplace = True)

In [61]:
test_corpus = []

for i in range(0, len(test_data)):
  news = re.sub('[^a-zA-Z0-9]', ' ', test_data['title'][i])
  news = news.lower()
  news = news.split()

  news = [lemmatizer.lemmatize(words) for words in news if not words in stopword ]
  news = ' '.join(news)

  test_corpus.append(news)

In [62]:
test_one_hot = [one_hot(word, voc_size) for word in test_corpus]

In [63]:
test_data['title'][1]

'Russian warships ready to strike terrorists near Aleppo'

In [64]:
test_corpus[1]

'russian warship ready strike terrorist near aleppo'

In [65]:
test_one_hot[1]

[2335, 18, 4354, 419, 1226, 3302, 191]

In [66]:
embedded_test = pad_sequences(test_one_hot, padding = 'post', maxlen = sent_length)
embedded_test

array([[1494, 3732, 3823, ...,    0,    0,    0],
       [2335,   18, 4354, ...,    0,    0,    0],
       [ 417, 3105, 2249, ...,    0,    0,    0],
       ...,
       [4177,  148, 3046, ...,    0,    0,    0],
       [3213, 3292, 4456, ...,    0,    0,    0],
       [ 254, 3771, 3963, ...,    0,    0,    0]], dtype=int32)

In [67]:
test_pred = model.predict(embedded_test)

163/163 [==============================] - 3s 6ms/step


In [68]:
test_pred = np.where(test_pred > 0.8, 1, 0)

In [69]:
test_pred

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [70]:

Test_pred = pd.DataFrame(test_pred)
Test_pred['id'] = test_data['id']
Test_pred.rename(columns = {0 : 'label'}, inplace = True)

In [71]:

Test_pred

,label,id
0,0,20800
1,1,20801
2,0,20802
3,0,20803
4,0,20804
...,...,...
5195,0,25995
5196,0,25996
5197,0,25997
5198,0,25998


***Saving the result***

In [72]:
Test_pred.to_csv("/content/drive/MyDrive/Fake News Datasets/Test_Pred.csv", index =False)

# ***Saving the Trained Model***

In [74]:
import pickle

#save the model
model_save_path = "/content/drive/MyDrive/Fake News Datasets/Fake News Classifier.h5"
model.save(model_save_path)

# save the model with pickel file
pickle_save_path = "/content/drive/MyDrive/Fake News Datasets/Fake News Classifier.pkl"
with open(pickle_save_path, 'wb') as f:
  pickle.dump(model_save_path, f)

  print(f'Model saved to {model_save_path}')
print(f'Model path saved to {pickle_save_path}')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to /content/drive/MyDrive/Fake News Datasets/Fake News Classifier.h5
Model path saved to /content/drive/MyDrive/Fake News Datasets/Fake News Classifier.pkl
